CC-BY license, c Samantha Horn and Sabina Sloman, 2022.

Reproduces results reported in Horn and Sloman (2022). A Comparison of Methods for Adaptive Experimentation. URL: <arXiv link>.
The contents of this notebook are licensed under a CC-BY-4.0 license."

In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)
import numpy as np
import pickle
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from scipy.stats import bernoulli, beta as betadist, sem

Mounted at /content/drive/


In [ ]:
ddir = "/content/drive/MyDrive/AdaptiveExperimentation"

In [ ]:
def get_treatment_prob(D, k):
  return np.array([ (D==i).mean() for i in np.arange(k) ])

def ThompsonSampling(alphas,betas,N):
  # Inputs: 
    # alphas: list of alphas for each treatment 
    # betas: list of bets for each treatment
    # N: total number of observation units
  # Output list of treatment assignments for each n in N and assignment probabilties 

  # for each n in N, draw a theta for each treatment and get max
  draws = betadist(a=alphas, b=betas).rvs(size=(N,3))
  D = np.argmax(draws, axis=1)

  return D, get_treatment_prob(D, len(alphas))

def ExplorationSampling(alphas,betas,N): # Kasy and Sautman 2021

  k = len(alphas) # of treatments
   
  # First get assignment probabilities under Simple Thompson sampling (ps)
  _ , ps = ThompsonSampling(alphas,betas,N)

  # Convert ps to qs - standardized p(1-p)
  q = ps*(1-ps)
  # If all qs are 0., assign randomly
  if sum(q) > 0:
    q /= q.sum()
  else:
    q += 1./k

  # Create assingmments based on these shares (flooring as in R code)
  shares = np.floor(q*N).astype(int)

  assgnmts = np.repeat(np.arange(k), shares)
  
  # Assign remainder randomnly
  remainder = np.random.randint(k, size=(N-shares.sum(),)) 
  assgnmts = np.append(assgnmts, remainder)

  return assgnmts, get_treatment_prob(assgnmts, k) # return list of assignments 

def RA(alphas,betas,N):
  return np.random.randint(len(alphas), size=(N,)), np.ones_like(alphas) / alphas.shape[0]

def TemperedThompson(alphas,betas,N,q=.2):
  mask = bernoulli.rvs(q, size=(N,)).astype(bool)
  assgnmts, _ = ThompsonSampling(alphas,betas,N)
  rand_assgnmts, _ = RA(alphas,betas,N)
  assgnmts[mask] = rand_assgnmts[mask]

  return assgnmts, get_treatment_prob(assgnmts, len(alphas)) # outputting assingment as list

def SimulateData(thetas,D):
  # Simulate Ys for each N for point T
  # Take into account N assigned to each treatment 
  # Take into account assumed and mean and variance of each treatment distribution
  # Inputs: thetas, D
  # Outputs: Data: Y_it for each i in N
  return bernoulli.rvs(thetas[D])

def UpdateAlphaBeta(alpha_0,beta_0,Data,assgnmts):
  # Calculates, n and s given data
  # Create class and store m and r (~ add n and s each round)
  # Update alpha and beta given results
  # Outpuuts: alphas and betas for posterior for each treatment

  # Assuming `Data` is a vector of binary outcomes, like the output of 
  # `SimulateData`
  alpha_1, beta_1 = alpha_0.copy(), beta_0.copy()
  for i in range(alpha_0.shape[0]):
    thesedata = Data[assgnmts==i]
    rt = thesedata.sum()
    mt = thesedata.shape[0]
    alpha_1[i] += rt
    beta_1[i] += mt-rt
  return alpha_1, beta_1 
  
###### Success metrics ######

def rmse(theta,alpha_hat,beta_hat):
  bias2 = (theta - SW(alpha_hat,beta_hat))**2
  var = (alpha_hat*beta_hat) / ((alpha_hat + beta_hat)**2 * (alpha_hat + beta_hat + 1))
  return np.sqrt(bias2 + var)

def SW(alpha,beta):
  return (alpha / (alpha + beta))

# All functions take the following inputs:
#   theta *np.array*: An array of length k that specifies the true effectiveness 
#     of each treatment.
#   alpha_hat *np.array*: An array of length k that specifies the estimated
#     alpha parameter corresponding to each treatment.
#   beta_hat *np.array*: An array of length k that specifies the estimated
#     beta parameter corresponding to each treatment.
#   Data *np.array*: A TxN array containing the outcomes for N participants
#     across T trials.
def InSampleRegret(theta,alpha_hat,beta_hat,Data):
  return theta.max() - Data.mean() 

def PolicyRegret(theta,alpha_hat,beta_hat,Data):
  theta_hat = SW(alpha_hat,beta_hat)
  return theta.max() - theta[theta_hat.argmax()]

def RMSEAvg(theta,alpha_hat,beta_hat,Data):
  return rmse(theta,alpha_hat,beta_hat).mean()

def RMSEBest(theta,alpha_hat,beta_hat,Data):
  theta_hat = SW(alpha_hat,beta_hat)
  return rmse(theta,alpha_hat,beta_hat)[theta_hat.argmax()]

def DistDiffTest(alpha_hat,beta_hat,p,N=10000):

    draws = betadist(a=alpha_hat, b=beta_hat).rvs(size=(N,3)) # draw samples

    # Get for T1 vs T2
    t1_t2_diff = (draws - draws[:,1,None]) 
    t1_t2_diff = np.delete(t1_t2_diff,0,1) 
    t1_t2_diff = np.delete(t1_t2_diff,0,1) 
    t1_t2_diff = np.where(t1_t2_diff>0,1,0).sum(axis = 0)/t1_t2_diff.shape[0]
    t1_t2_correct = t1_t2_diff > (1-p)

    # Get vs control
    diff = (draws - draws[:,0,None]) # subtract first col from all 
    diff = np.delete(diff,0,1) # remove first col
    diff = np.where(diff>0,1,0).sum(axis = 0)/diff.shape[0] # get percent where diff is > 0 

    best_correct = diff[1] > (1-p) # get if largest treatment is correct
    secbest_correct = diff[0] > (1-p) # get if second largest treatment is correct

    pc_correct = (t1_t2_correct.astype(int) + best_correct.astype(int) + secbest_correct.astype(int)) / alpha_hat.shape[0]

    # Make binary
    bin_correct = np.where(pc_correct != 1,0,1)

    return 1-bin_correct

In [ ]:
k = 3

def AdaptiveLoop(alpha_0s,beta_0s,N,iterations,method,thetas):

  # Simulate initial data and start with random assigment 
  
  alpha, beta = alpha_0s.copy(), beta_0s.copy()
  D , assgmnt_probs = method(alpha,beta,N)
  data = SimulateData(thetas,D)
  Data = np.array([data])
  alpha, beta = UpdateAlphaBeta(alpha,beta,data,D)
  for t in range(iterations-1):
    D , assgmnt_probs = method(alpha,beta,N)
    data = SimulateData(thetas,D)
    Data = np.concatenate((Data,data[None,:]))
    alpha, beta = UpdateAlphaBeta(alpha,beta,data,D)

  return alpha,beta,Data

In [ ]:
def sweep(N, Nwaves, all_thetas,p):
  ntheta = all_thetas.shape[0]

  ISRegret = np.full((len(Nwaves),ntheta,4,nreps), fill_value=np.nan)
  PRegret = ISRegret.copy()
  RMSEAv = ISRegret.copy()
  RMSEBe = ISRegret.copy()
  SigDiffAll = ISRegret.copy()

  for i,nt in enumerate(Nwaves):
    nparticipants = int(N / nt)
    for j in range(ntheta):
      these_thetas = all_thetas[j,:]

      for k,method in enumerate([
        ThompsonSampling,ExplorationSampling,TemperedThompson,RA
      ]):
        print(f"{i*ntheta*4+j*4+k} / {len(Nwaves)*ntheta*4} done")

        for l in range(nreps):
          alpha,beta,Data = AdaptiveLoop(
            alpha_0s,beta_0s,nparticipants,nt,method,these_thetas
          )

          ISRegret[i,j,k,l] = InSampleRegret(these_thetas,alpha,beta,Data)
          PRegret[i,j,k,l] = PolicyRegret(these_thetas,alpha,beta,Data)
          RMSEAv[i,j,k,l] = RMSEAvg(these_thetas,alpha,beta,Data)
          RMSEBe[i,j,k,l] = RMSEBest(these_thetas,alpha,beta,Data)
          SigDiffAll[i,j,k,l] = DistDiffTest(alpha,beta,p)

  return ISRegret, PRegret, RMSEAv, RMSEBe, SigDiffAll

In [ ]:
# Looping over multiple scenarios
alpha_0s = np.ones((3,))
beta_0s = np.ones((3,))
all_thetas = np.random.uniform(size=(10000,3))
all_thetas = np.sort(all_thetas, axis=1)
Nwaves = [1,10,100,250]
nreps = 1
p = 0.05

# ISRegret, PRegret, RMSEAv, RMSEBe, SigDiffAll, SigDiffBest, SigDiffSecBest = sweep(1000, Nwaves, all_thetas,p)

# Save Files

# with open(f"{ddir}/ISRegret", "wb") as wfh:
#   pickle.dump(ISRegret, wfh)
# with open(f"{ddir}/PRegret", "wb") as wfh:
#   pickle.dump(PRegret, wfh)
# with open(f"{ddir}/RMSEAv", "wb") as wfh:
#   pickle.dump(RMSEAv, wfh)
# with open(f"{ddir}/RMSEBe", "wb") as wfh:
#   pickle.dump(RMSEBe, wfh)
# with open(f"{ddir}/SigDiffAll", "wb") as wfh:
#   pickle.dump(SigDiffAll, wfh)

In [ ]:
# Load Files

ISRegret = pickle.load(open(f"{ddir}/ISRegret", "rb"))
PRegret = pickle.load(open(f"{ddir}/PRegret", "rb"))
RMSEAv = pickle.load(open(f"{ddir}/RMSEAv", "rb"))**.5
RMSEBe = pickle.load(open(f"{ddir}/RMSEBe", "rb"))**.5
SigDiffAll = pickle.load(open(f"{ddir}/SigDiffAll", "rb"))

In [ ]:
colors = ["blue","green","dimgray","deepskyblue"]
rgb = np.array([[0.,0.,255.],[0.,128.,0.],[105.,105.,105.],[0.,191.,255.]])

In [ ]:
def linesubplot(all_data,titles,Nwaves,names,rows,cols,fontsize=12):
  groups = ['group1','group2','group3','group4']

  fig = make_subplots(rows=rows, cols=cols, subplot_titles=titles, vertical_spacing = 0.15)
  fig.update_layout(font_size=fontsize)
  fig.update_annotations(font_size=fontsize)

  r = 1
  c = 1
  for d in range(len(all_data)):

    stat = all_data[d]
    title = titles[d]
    data = stat[1:4,:,:,0].mean(axis=1)
    se = 1.96*sem(stat[1:4,:,:,0], axis=1)

    if d == 0:
      leg = True
    else: 
      leg = False

    for ii in range(0,4):

      fig.add_trace(go.Scatter(
        name = names[ii],
        x=Nwaves, y=data[:,ii], 
        error_y=dict(
            type="data", 
            array=se[:,ii],
            visible=True), 
        legendgroup=groups[ii],
        line_color=colors[ii],
        showlegend=leg),
        row = r, col = c)

    fig.update_layout(
      autosize=False,
      width=cols*500,
      height=rows*500)
    
    fig.update_layout(legend=dict(
      yanchor="top",
      y=-0.1,
      xanchor="center",
      x=0.5, orientation="h"
      ))
    
    fig.update_xaxes(title_text="Experimental Waves", title_font_size=fontsize)

    if c == cols :
      r += 1
    c += 1
    if c == cols + 1:
      c = 1

  return fig



In [ ]:
N_list = ['10','100','250']
names= ["Thompson","Exploration","Tempered Thompson","RA"]

all_data = [ISRegret,PRegret,RMSEAv,RMSEBe,SigDiffAll]
titles = ['In-Sample Regret','Policy Regret','MSE Average','MSE Best','Identified Correct Order']
titles = ["Panel A: <i>R<sub>sample</sub><i>","Panel B: <i>R<sub>policy</sub><i>","Panel C: <i>PREC<sub>avg</sub><i>","Panel D: <i>PREC<sub>best</sub><i>","Panel E: <i>SP<i>"]

fig = linesubplot(all_data,titles,N_list,names,2,3,22)
fig.show()

In [ ]:
losses = [ISRegret,RMSEBe,PRegret,RMSEAv,SigDiffAll]
nmeasures = len(losses)
loss_cats = ["regret","prec","regret","prec","power"]
loss_labs = [
  "<i>R<sub>sample</sub><i>","<i>PREC<sub>best</sub><i>",
  "<i>R<sub>policy</sub><i>","<i>PREC<sub>avg</sub><i>","<i>SP<i>"
]
fig = make_subplots(rows=1, cols=3, subplot_titles=[
  "Panel A: 10 waves (<i>N<sub>t</sub><i> = 100)",
  "Panel B: 100 waves (<i>N<sub>t</sub><i> = 10)",
  "Panel C: 250 waves (<i>N<sub>t</sub><i> = 4)"
])

for nwaves_idx in range(1,4):
  Z = np.full((nmeasures,nmeasures,4), fill_value=255.)
  texti = np.zeros((nmeasures,nmeasures))
  for i, loss1 in enumerate(losses):
    for j, loss2 in enumerate(losses):
      if j < i:
        continue
      cat1 = loss_cats[i]
      cat2 = loss_cats[j]
      l1 = loss1[nwaves_idx,:,:,:]
      l2 = loss2[nwaves_idx,:,:,:]
      if (cat1 == "power") and (cat2 == "power"):
        hybrid_loss = ( l1 + l2 ) / 2.
      elif cat1 == "power":
        hybrid_loss = l1.copy()
        hybrid_loss[hybrid_loss == 0.] = l2[hybrid_loss == 0.]
      elif cat2 == "power":
        hybrid_loss = l2.copy()
        hybrid_loss[hybrid_loss == 0.] = l1[hybrid_loss == 0.]
      else:
        hybrid_loss = ( l1 + l2 ) / 2.
      sorted_each = np.argsort(hybrid_loss[:,:,0], axis=1)
      which, cts = np.unique(sorted_each[:,0], return_counts=True)
      Z[i,j,:-1] = rgb[which[cts.argmax()]]
      texti[i,j] = cts.max() / cts.sum()
  Z[:,:,-1] = 150.
  fig.add_trace(px.imshow(
      Z, x=np.arange(nmeasures), y=np.arange(nmeasures)
  )["data"][0], row=1, col=nwaves_idx)

  for i in range(nmeasures):
    for j in range(nmeasures):
      if j < i:
        continue
      fig.add_annotation(
          x=j, y=i, text=( str(np.round_(texti[i,j], 2))+"0" )[1:4], 
          showarrow=False, row=1, col=nwaves_idx
      )

for i, lab in enumerate((
    "Thompson", "Exploration", "Tempered Thompson", "RA"
)):
  fig.add_trace(go.Scatter(
    x=[np.nan,np.nan], y=[np.nan,np.nan], mode="markers", marker=dict(
      color=f"rgba({','.join(rgb[i,:].astype(str))},.59)", size=100., 
      symbol="square"
    ), name=lab
), row=1, col=3)
fig.update_layout(legend=dict(orientation="h", x=.25, y=-.3))

fig.update_layout(
    font=dict(size=15), plot_bgcolor="white", height=500., width=1200.
)
fig.update_xaxes(ticktext=loss_labs, tickvals=np.arange(nmeasures))
fig.update_yaxes(ticktext=loss_labs, tickvals=np.arange(nmeasures))